In [2]:
import pandas as pd
import numpy as np

In [3]:
df_train = pd.read_csv('../resources/train.csv', delimiter='|')
df_predict = pd.read_csv('../resources/predict_case.csv', delimiter='|')

In [4]:
df_train.drop(['id', 'job_description'], axis=1, inplace=True)
df_predict.drop(['job_description'], axis=1, inplace=True)

In [5]:
def cumulative_categorize(df, column, threshold):
    df[column] = df[column].astype(str)
    counts = df[column].value_counts()
    df[column] = df[column].apply(lambda x: x if counts[x] > threshold else 'Other')
    return df

In [6]:
# clean experience_level
mode_experience_level = df_train['experience_level'].mode()[0]
df_train['experience_level'] = df_train['experience_level'].fillna(mode_experience_level)

df_train = cumulative_categorize(df_train, 'experience_level', 50)

# clean education_levels
unique_values = set(df_train['education_level'].unique()) - set(df_predict['education_level'].unique())
df_train.loc[df_train['education_level'].isin(unique_values), 'education_level'] = 'Other'

# clean employment_type
mode_employment_type = df_train['employment_type'].mode()[0]
df_train['employment_type'] = df_train['employment_type'].fillna(mode_employment_type)

df_train = cumulative_categorize(df_train, 'employment_type', 50)

# clean job_function
unique_values = set(df_train['job_function'].unique()) - set(df_predict['job_function'].unique())
df_train.loc[df_train['job_function'].isin(unique_values), 'job_function'] = 'Other'

# clean job_benefits
mode_job_benefits = df_train['job_benefits'].mode()[0]
df_train['job_benefits'] = df_train['job_benefits'].fillna(mode_job_benefits)

unique_values = set(df_train['job_benefits'].unique()) - set(df_predict['job_benefits'].unique())
df_train.loc[df_train['job_benefits'].isin(unique_values), 'job_benefits'] = 'Other'

# clean company_process_time
mode_company_process_time = df_train['company_process_time'].mode()[0]
df_train['company_process_time'] = df_train['company_process_time'].fillna(mode_company_process_time)

unique_values = set(df_train['company_process_time'].unique()) - set(df_predict['company_process_time'].unique())
df_train.loc[df_train['company_process_time'].isin(unique_values), 'company_process_time'] = 'Other'

# clean company_size
mode_company_size = df_train['company_size'].mode()[0]
df_train['company_size'] = df_train['company_size'].fillna(mode_company_size)

unique_values = set(df_train['company_size'].unique()) - set(df_predict['company_size'].unique())
df_train.loc[df_train['company_size'].isin(unique_values), 'company_size'] = 'Other'

# clean company_industry
mode_company_industry = df_train['company_industry'].mode()[0]
df_train['company_industry'] = df_train['company_industry'].fillna(mode_company_industry)

unique_values = set(df_train['company_industry'].unique()) - set(df_predict['company_industry'].unique())
df_train.loc[df_train['company_industry'].isin(unique_values), 'company_industry'] = 'Other'

# clean salary and salary currency
mode_salary_currency = df_train['salary_currency'].mode()[0]
df_train['salary_currency'].fillna(mode_salary_currency, inplace=True)
mean_idr_salary = df_train[df_train['salary_currency'] == 'IDR']['salary'].mean()
mean_usd_salary = df_train[df_train['salary_currency'] == 'USD']['salary'].mean()

df_train.loc[(df_train['salary_currency'] == 'IDR') & (df_train['salary'].isnull()), 'salary'] = mean_idr_salary
df_train.loc[(df_train['salary_currency'] == 'USD') & (df_train['salary'].isnull()), 'salary'] = mean_usd_salary

C:\Users\PC\AppData\Local\Temp\ipykernel_20640\466768370.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['salary_currency'].fillna(mode_salary_currency, inplace=True)


In [7]:
# clean job_title
import json
import re

with open('../resources/library.json') as f:
    library = json.load(f)

stopwords = set(['a', 'an', 'the', 'in', 'on', 'at', 'to', 'for', 'with', 'by', 'of', 'and', 'or', 'bidang'])

def clean_job_title(text):
    if (text == 'Other'):
        return text
    text = text.lower()

    words = text.split()
    new_title = ''
    prev_word = ''
    for word in words:
        if word == 'to':
            new_title += word.upper() + ' '
            continue
        if word in stopwords:
            continue
        if word == "front" or word == "back":
            prev_word = word
            continue
        if word == "end":
            if prev_word == "front" or prev_word == "back":
                new_title += prev_word.capitalize() + '' + word.capitalize() + ' '
                prev_word = ''
            continue
        for category, keywords in library.items():
            if word in keywords:
                if category not in new_title:
                    new_title += category + ' '
    if not new_title:
        new_title = 'Other'
    return new_title.strip()

# df_new = df_train['job_title'].apply(clean_job_title)
# df_new.to_csv('../resources/job_title_category.csv', index=False)
df_train['job_title'] = df_train['job_title'].apply(clean_job_title)

# df_new_2 = df_predict['job_title'].apply(clean_job_title)
# df_new_2.to_csv('../resources/job_title_category_predict.csv', index=False)
df_predict['job_title'] = df_predict['job_title'].apply(clean_job_title)

In [8]:
# clean experience_level
mode_experience_level = df_predict['experience_level'].mode()[0]
df_predict['experience_level'] = df_predict['experience_level'].fillna(mode_experience_level)

# clean job benefits
mode_job_benefits = df_predict['job_benefits'].mode()[0]
df_predict['job_benefits'] = df_predict['job_benefits'].fillna(mode_job_benefits)

# clean company_process_time
mode_company_process_time = df_predict['company_process_time'].mode()[0]
df_predict['company_process_time'] = df_predict['company_process_time'].fillna(mode_company_process_time)

# clean company_size
mode_company_size = df_predict['company_size'].mode()[0]
df_predict['company_size'] = df_predict['company_size'].fillna(mode_company_size)

# clean company_industry
mode_company_industry = df_predict['company_industry'].mode()[0]
df_predict['company_industry'] = df_predict['company_industry'].fillna(mode_company_industry)


# clean job_title
df_predict['job_title'] = df_predict['job_title'].apply(clean_job_title)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

X_train = df_train.drop('salary', axis=1)
X_test = df_predict.drop('id', axis=1)
y_train = df_train['salary']

vectorizer = CountVectorizer()
job_title_encoded = vectorizer.fit_transform(df_train['job_title'])

job_title_encoded_df = pd.DataFrame(job_title_encoded.toarray(), columns=vectorizer.get_feature_names_out())
X_train = pd.concat([X_train, job_title_encoded_df], axis=1)
X_train.drop('job_title', axis=1, inplace=True)

job_title_predict_encoded = vectorizer.transform(df_predict['job_title'])
job_title_predict_encoded_df = pd.DataFrame(job_title_predict_encoded.toarray(), columns=vectorizer.get_feature_names_out())
X_test = pd.concat([X_test, job_title_predict_encoded_df], axis=1)

categorical_features = ['experience_level', 'education_level', 'employment_type', 'job_function', 'job_benefits', 'company_process_time', 'company_size', 'company_industry', 'salary_currency']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])

pipeline.fit(X_train, y_train)
Y_predict = pipeline.predict(X_test)

# from sklearn.metrics import mean_squared_error
# print(mean_squared_error(y, y_pred))
# print(pipeline.score(X, y)*100)


In [14]:
# write csv id and salary
df_predict['salary'] = Y_predict
mean_idr_salary = df_predict[df_predict['salary_currency'] == 'IDR']['salary'].mean()
mean_usd_salary = df_predict[df_predict['salary_currency'] == 'USD']['salary'].mean()
df_predict.loc[(df_predict['salary_currency'] == 'IDR') & (df_predict['salary'].isnull()), 'salary'] = mean_idr_salary
df_predict.loc[(df_predict['salary_currency'] == 'USD') & (df_predict['salary'].isnull()), 'salary'] = mean_usd_salary
df_predict[['id', 'salary']].to_csv('../resources/predicted_salary.csv', index=False)
